In [70]:
DL = '/Users/damiansp/Downloads'

In [71]:
!rm $DL/*

In [80]:
from datetime import datetime
import os

import numpy as np
import pandas as pd

In [105]:
SYMBOLS = [
    'AAPL', 'ABMD', 'ABNB', 'AMD', 'AMZN', 'ANET', 'APPH', 'AXON',
    'BROS', 'BYND', 'CHWY', 'COIN', 'COUR', 'CRWD', 'DDOG', 'DKNG',
    'DOCU', 'DUOL', 'EDIT', 'ETSY', 'FLGT', 'FUV', 'FVRR', 'GH', 'GMED',
    'GOOG', 'INTG', 'ISRG', 'JD', 'LSPD', 'LULU','MDB', 'MELI', 'MNST',
    'MSFT', 'MTCH', 'NVCR', 'NVDA', 'NYT', 'OKTA', 'OPEN', 'PANW',
    'PINS', 'PTON', 'QDEL', 'RBLX', 'RDFN', 'RGEN', 'ROKU', 'SE',
    'SHOP', 'SPLK', 'SQ', 'TASK', 'TDOC', 'TEAM', 'TREX', 'TTD', 'TTWO',
    'TWLO', 'TWTR', 'UPST', 'VEEV', 'WK', 'XPEV', 'ZEN', 'ZNGA']

In [106]:
!ls $DL

Holdings - Damian Satterthwaite-Phillips(1).csv
Holdings - Damian Satterthwaite-Phillips(2).csv
Holdings - Damian Satterthwaite-Phillips.csv
Portfolio_Positions_Mar-25-2022.csv
Positions.csv
mw.csv


In [107]:
def check_symbols(df):
    all_present = True
    for symbol in df.index.unique():
        if symbol not in SYMBOLS:
            print(f'{symbol} not in SYMBOLS')
            all_present = False
    if all_present:
        print('All clear')

In [108]:
def convert_value(s):
    return round(float(s.replace('$', '').replace(',', '')))

In [109]:
binder = pd.DataFrame(index=SYMBOLS)
binder.head()

""
AAPL
ABMD
ABNB
AMD
AMZN


### E*Trade

In [110]:
DONGMEI = False

In [117]:
if DONGMEI:
    et_df = (
        pd.read_csv(
            f'{DL}/Positions.csv', index_col=0, skiprows=1
        )[['Market Value']]
        .rename(columns={'Market Value': 'ET'}))
    et_df.index = map(lambda x: x.split()[0], et_df.index)
    et_df.drop(index=['Portfolio', 'Cash'], inplace=True)
else:
    et_df = (
        pd.read_csv(
            f'{DL}/Positions.csv', index_col=0, skiprows=1
        )[['Market Value']]
        .rename(columns={'Market Value': 'ET'}))
    et_df.index = map(lambda x: x.split()[0], et_df.index)
    et_df.drop(index=['FNRG', 'Portfolio', 'Cash'], inplace=True)
    #et_df.ET = et_df.ET.apply(lambda x: round(float(x)))
et_df.head()

,ET
AAPL,174.72
ABMD,2232.16
ABNB,1337.76
AMZN,3295.47
ANET,276.8


In [120]:
et_df.loc[et_df.ET == '--']
et_df.loc['TREX', 'ET'] = 68.96

In [121]:
check_symbols(et_df)

All clear


In [122]:
out = pd.concat([binder, et_df], axis=1).fillna(0)
out.ET = out.ET.astype(float).apply(lambda x: int(round(x)))
out.head()

,ET
AAPL,175
ABMD,2232
ABNB,1338
AMD,0
AMZN,3295


### Fidelity

In [123]:
TODAY = datetime.now().date()
TODAY
today = datetime.strftime(TODAY, '%b-%d-%Y')
filename = f'Portfolio_Positions_{today}.csv'
filename

'Portfolio_Positions_Mar-25-2022.csv'

In [124]:
df_raw = pd.read_csv(f'{DL}/{filename}')
df_raw.head()

,Account Number,Account Name,Symbol,Description,Quantity,Last Price,Last Price Change,Current Value,Today's Gain/Loss Dollar,Today's Gain/Loss Percent,Total Gain/Loss Dollar,Total Gain/Loss Percent,Percent Of Account,Cost Basis,Cost Basis Per Share,Type
0,218520300,ROLLOVER IRA,SPAXX**,FIDELITY GOVERNMENT MONEY MARKET,140.4,$1.00,$0.00,$140.40,NaN,NaN,NaN,NaN,0.29%,NaN,NaN,Cash
1,218520300,ROLLOVER IRA,AAPL,APPLE INC,16.0,$174.72,+$0.65,$2795.52,+$10.40,+0.37%,+$371.36,+15.31%,5.79%,$2424.16,$151.51,Cash
2,218520300,ROLLOVER IRA,ABMD,ABIOMED INC COM USD0.01,1.0,$318.88,-$2.59,$318.88,-$2.59,-0.81%,+$33.02,+11.55%,0.66%,$285.86,$285.86,Cash
3,218520300,ROLLOVER IRA,ABNB,AIRBNB INC COM CL A,3.0,$167.22,-$3.61,$501.66,-$10.83,-2.12%,+$73.41,+17.14%,1.04%,$428.25,$142.75,Cash
4,218520300,ROLLOVER IRA,AMD,ADVANCED MICRO DEVICES INC,10.0,$119.67,-$0.86,$1196.70,-$8.60,-0.72%,+$171.36,+16.71%,2.48%,$1025.34,$102.53,Cash


In [125]:
df = df_raw[['Account Name', 'Symbol', 'Current Value']].dropna()
df['Current Value'] = df['Current Value'].apply(convert_value)
df['Current Value'] = df['Current Value'].fillna(0).astype(int)
df = df[((df.Symbol != 'SPAXX**') & (df.Symbol != 'Pending Activity'))]
df.head()

,Account Name,Symbol,Current Value
1,ROLLOVER IRA,AAPL,2796
2,ROLLOVER IRA,ABMD,319
3,ROLLOVER IRA,ABNB,502
4,ROLLOVER IRA,AMD,1197
5,ROLLOVER IRA,ANET,1522


In [126]:
accounts = {account_name: df[df['Account Name'] == account_name]
            for account_name in df['Account Name'].unique()}
for acct in accounts.keys():
    accounts[acct].index = accounts[acct].Symbol
    accounts[acct] = (
        accounts[acct]
        .rename(columns={'Current Value': acct})
        .drop(columns=['Account Name', 'Symbol']))
    
accounts['ROLLOVER IRA'].head()

,ROLLOVER IRA
Symbol,
AAPL,2796
ABMD,319
ABNB,502
AMD,1197
ANET,1522


In [127]:
for k, v in accounts.items():
    print(k)
    check_symbols(v)
    print()

ROLLOVER IRA
All clear

ROTH IRA
All clear

SIMPLE IRA
All clear



In [128]:
for v in accounts.values():
    out = pd.concat([out, v], axis=1)
out.fillna(0, inplace=True)
out = out.astype(int)
out.head()

,ET,ROLLOVER IRA,ROTH IRA,SIMPLE IRA
AAPL,175,2796,2097,0
ABMD,2232,319,638,0
ABNB,1338,502,167,0
AMD,0,1197,6223,0
AMZN,3295,0,3295,0


In [129]:
out.to_csv(f'{DL}/clean.csv')

In [130]:
!open $DL/clean.csv

In [34]:
!rm $DL/$filename $DL/Positions.csv $DL/clean.csv
!ls $DL

Holdings - Damian Satterthwaite-Phillips.csv


### MarketWatch

In [79]:
!ls $DL

Holdings - Damian Satterthwaite-Phillips(1).csv
Holdings - Damian Satterthwaite-Phillips(2).csv
Holdings - Damian Satterthwaite-Phillips.csv


In [96]:
dfs = [binder]
for i, f in enumerate(sorted(os.listdir(DL))):
    print(f)
    if f.startswith('Holdings'):
        mw = pd.read_csv(f'{DL}/{f}')[['Symbol', 'Value']]
        mw.rename(columns={'Value': f'Value_{i}'}, inplace=True)
        mw.index = mw.Symbol
        mw.drop('Symbol', axis=1, inplace=True)
        mw[f'Value_{i}'] = mw[f'Value_{i}'].apply(convert_value)
        dfs.append(mw)
mw = pd.concat(dfs, axis=1).fillna(0).astype(int)
mw.head()

Holdings - Damian Satterthwaite-Phillips(1).csv
Holdings - Damian Satterthwaite-Phillips(2).csv
Holdings - Damian Satterthwaite-Phillips.csv


,Value_0,Value_1,Value_2
AAPL,3145,0,874
ABMD,957,0,638
ABNB,167,0,1003
AMD,2274,2034,838
AMZN,3295,0,0


In [98]:
check_symbols(mw)

All clear


In [100]:
#out = pd.concat([binder, mw], axis=1).fillna(0).astype(int)
mw.head()

,Value_0,Value_1,Value_2
AAPL,3145,0,874
ABMD,957,0,638
ABNB,167,0,1003
AMD,2274,2034,838
AMZN,3295,0,0


In [103]:
mw.to_csv(f'{DL}/mw.csv')

In [104]:
!open $DL/mw.csv

In [18]:
!rm $DL/"Holdings - Damian Satterthwaite-Phillips.csv" $DL/mw.csv
!ls $DL

In [15]:
!rm $DL/*